### Equipo 2

**Rodríguez Fitta Emanuel**

**Ramos Corona Leonardo Alfonso**


#### Tarea

Se tiene un archivo (dates.txt) donde cada línea de este corresponde a una nota médica y cada nota tiene una fecha que debe extraerse, pero cada fecha está codificada en uno de muchos formatos. Por ejemplo, se muestra a continuación una lista de algunas de las variantes que se puede encontrar en este conjunto de datos:
<ul>
    <li>04/20/2009; 04/20/09; 4/20/09; 4/3/09</li>
    <li>Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;</li>
    <li>20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009</li>
    <li>Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009</li>
    <li>Feb 2009; Sep 2009; Oct 2010</li>
    <li>6/2008; 12/2009</li>
    <li>2009; 2010</li>
</ul>
La actividad consiste en:
<p>a). Identificar correctamente todas las diferentes variantes de fecha codificadas en este conjunto de datos, normalizar y ordenar adecuadamente las fechas.</p>
<p>b).	Una vez que haya extraído estos patrones de fecha del texto, el siguiente paso es clasificarlos en orden cronológico ascendente de acuerdo con las siguientes reglas:</p>
<ul>
    <li>Todas las fechas están en formato xx/xx/xx son mm/dd/aa</li>
    <li>Todas las fechas en las que el año está codificado en solo dos dígitos corresponden a años posteriores a la década de 1900 (p. Ej., 1/5/89 es el 5 de enero de 1989).</li>
    <li>Si falta el día (p. Ej., 9/2009), suponga que es el primer día del mes (p. Ej., septiembre, 1 de 2009).</li>
    <li>Si falta el mes (por ejemplo, 2010), suponga que es el primero de enero de ese año (p. Ej., enero, 1 de 2010).</li>
    <li>Tenga cuidado con los posibles errores tipográficos, ya que este es un conjunto de datos derivados de la vida real.</li>
</ul>
Esta función debería devolver una lista de longitud 500.

### Importante: Generamos dos funciones con diferentes técnicas de aplicación de expresiones regulares y ambas nos generaron los mismos resultados ordenados

## Función 1. En esta función utilizamos un pandas dataframe como contenedor para controlar las expresiones regulares. Creamos etiquetas de día, mes y año que nos permiten finalmente generar las fechas depuradas. El resultado se exporta a un archivo scv separado por comas.

In [15]:
import pandas as pd
import re

In [16]:
# Guardamos la información en un pandas dataframe
with open('dates.txt') as archivo:
    df = pd.Series(archivo.readlines())

df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [17]:
# Función de obtención de fechas
def obtener_fechas_ordenadas():
    # Obtenemos las fechas y las almacenamos en un pandas dataframe
    # A la expresión regular le agregamos las etiquetas de f_origen (fecha de origen), dia, mes y año
    # Con este patrón solo se extrajeron 25 fechas
    fechas_extraidas = df.str.extractall(r'(?P<f_origen>(?P<mes>\d?\d)[/|-](?P<dia>\d?\d)[/|-](?P<anio>\d{4}))')
    
    # df.index.isin: Devuelve una matriz booleana donde los valores del índice están en valores.
    # Generamos un arreglo de índices con los valores booleanos negados que nos indica las fechas que ya fueron procesadas
    # Con este patrón se obtuvieron 125 fechas
    indice_i = ~df.index.isin([x[0] for x in fechas_extraidas.index])
    fechas_extraidas = fechas_extraidas.append(df[indice_i].str.extractall(r'(?P<f_origen>(?P<mes>\d?\d)[/|-](?P<dia>([0-2]?[0-9])|([3][01]))[/|-](?P<anio>\d{2}))'))
    indice_i = ~df.index.isin([x[0] for x in fechas_extraidas.index])
    del fechas_extraidas[3] # Eliminamos las columnas 3 y 4 agregadas
    del fechas_extraidas[4]
    
    # Agregamos los patrones nombres de meses con caracteres
    # Con este patrón recuperamos 230 fechas
    fechas_extraidas = fechas_extraidas.append(df[indice_i].str.extractall(r'(?P<f_origen>(?P<dia>\d?\d) ?(?P<mes>[a-zA-Z]{3,})\.?,? (?P<anio>\d{4}))'))
    indice_i = ~df.index.isin([x[0] for x in fechas_extraidas.index])
    fechas_extraidas = fechas_extraidas.append(df[indice_i].str.extractall(r'(?P<f_origen>(?P<mes>[a-zA-Z]{3,})\.?-? ?(?P<dia>\d\d?)(th|nd|st)?,?-? ?(?P<anio>\d{4}))'))
    del fechas_extraidas[3]
    indice_i = ~df.index.isin([x[0] for x in fechas_extraidas.index])

    
    # Fechas sin día
    # Agregando este patrón localizmos 457 fechas
    fechas_sin_dia = df[indice_i].str.extractall('(?P<f_origen>(?P<mes>[A-Z][a-z]{2,}),?\.? (?P<anio>\d{4}))')
    fechas_sin_dia = fechas_sin_dia.append(df[indice_i].str.extractall(r'(?P<f_origen>(?P<mes>\d\d?)/(?P<anio>\d{4}))'))
    fechas_sin_dia['dia'] = 1
    fechas_extraidas = fechas_extraidas.append(fechas_sin_dia)
    indice_i = ~df.index.isin([x[0] for x in fechas_extraidas.index])

    # Solo año
    # Agregando este patrón localizmos las 500 fechas del documento
    fechas_solo_anio = df[indice_i].str.extractall(r'(?P<f_origen>(?P<anio>\d{4}))')
    fechas_solo_anio['dia'] = 1
    fechas_solo_anio['mes'] = 1
    fechas_extraidas = fechas_extraidas.append(fechas_solo_anio)
    indice_i = ~df.index.isin([x[0] for x in fechas_extraidas.index])

    # Homologamos fechas dejándolas en un formato legible de mm/dd/aaaa
    # La función pd.to_datetime la convierte a dd/mm/aaaa tomando los parámetros del sistema operativo
    # Año
    fechas_extraidas['anio'] = fechas_extraidas['anio'].apply(lambda x: '19' + x if len(x) == 2 else x)
    fechas_extraidas['anio'] = fechas_extraidas['anio'].apply(lambda x: str(x))

    # Mes
    fechas_extraidas['mes'] = fechas_extraidas['mes'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    mes_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
    fechas_extraidas.replace({"mes": mes_dict}, inplace=True)
    fechas_extraidas['mes'] = fechas_extraidas['mes'].apply(lambda x: str(x))

    # Día
    fechas_extraidas['dia'] = fechas_extraidas['dia'].apply(lambda x: str(x))

    # Fecha depurada
    fechas_extraidas['fecha_depurada'] = fechas_extraidas['mes'] + '/' + fechas_extraidas['dia'] + '/' + fechas_extraidas['anio']
    fechas_extraidas['fecha_depurada'] = pd.to_datetime(fechas_extraidas['fecha_depurada'])

    # Ordenamos los registros de manera cronológica ascendente por medio de la fecha depurada
    fechas_extraidas.sort_values(by='fecha_depurada', inplace=True)
    return fechas_extraidas

In [18]:
datos = obtener_fechas_ordenadas()
datos.head()

,,f_origen,mes,dia,anio,fecha_depurada
,match,,,,,
9,0,4/10/71,4,10,1971,1971-04-10
84,0,5/18/71,5,18,1971,1971-05-18
2,0,7/8/71,7,8,1971,1971-07-08
53,0,7/11/71,7,11,1971,1971-07-11
28,0,9/12/71,9,12,1971,1971-09-12


In [19]:
datos['fecha_depurada'].count()

500

In [20]:
# Guardamos los resultados en un archivo de excel
datos.to_csv('fechas_ordenadas.csv', index=False)

#### Se recuperaron 500 fechas desde 1971 hasta 2016

In [21]:
# Recuperamos los registros del csv generado para comprobar.
df = pd.read_csv('fechas_ordenadas.csv', sep=',')

# Creamos el dataframe
df = pd.DataFrame(df)
    
df.head()

,f_origen,mes,dia,anio,fecha_depurada
0,4/10/71,4,10,1971,1971-04-10
1,5/18/71,5,18,1971,1971-05-18
2,7/8/71,7,8,1971,1971-07-08
3,7/11/71,7,11,1971,1971-07-11
4,9/12/71,9,12,1971,1971-09-12


In [22]:
df.tail()

,f_origen,mes,dia,anio,fecha_depurada
495,5/2016,5,1,2016,2016-05-01
496,30 May 2016,5,30,2016,2016-05-30
497,13 Oct 2016,10,13,2016,2016-10-13
498,19 Oct 2016,10,19,2016,2016-10-19
499,11/2016,11,1,2016,2016-11-01


## Función 2. En esta función utilizamos la librería re y sus métodos para controlar las expresiones regulares. El resultado se muestra en una lista.

In [43]:
import re
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [47]:
def transforma_mes(x):
  if x[0:3].lower() == 'jan':
    return 1
  elif x[0:3].lower() == 'feb':
    return 2
  elif x[0:3].lower() == 'mar':
    return 3
  elif x[0:3].lower() == 'apr':
    return 4
  elif x[0:3].lower() == 'may':
    return 5
  elif x[0:3].lower() == 'jun':
    return 6
  elif x[0:3].lower() == 'jul':
    return 7
  elif x[0:3].lower() == 'aug':
    return 8
  elif x[0:3].lower() == 'sep':
    return 9
  elif x[0:3].lower() == 'oct':
    return 10
  elif x[0:3].lower() == 'nov':
    return 11
  elif x[0:3].lower() == 'dec':
    return 12

In [50]:
def obtener_fechas(archivo):
  try:
    f = open(archivo + '.txt', 'r')
    dates = f.read() # Se guarda el texto en la variable texto
    print('Archivo cargado correctamente')
  except: # En caso de ingresar un nombre incorrecto se imprime un mensaje y termina la función
    print('Archivo no encontrado')

  # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
  fechas1 = re.findall(r'(?:\d{2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]? [12]\d{3}', dates, flags = re.I) # Regexp
  fechas1_ = [fecha.split(' ') for fecha in fechas1] # Separación de cada fecha por espacios
  fechas1_ = [[1] + fecha_list  if len(fecha_list) == 2 else fecha_list for fecha_list in fechas1_  ] # Colocamos el primer día de mes a aquellas fechas que no tienen específicado el día
  fechas1_ = pd.DataFrame(fechas1_, columns = ['day', 'month', 'year']) # Creación del dataframe con estas fechas
  fechas1_['month'] = fechas1_['month'].apply(transforma_mes) # Transforrmamos los meses a número

  # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
  fechas2 = re.findall('(\d{1,2}[/-]\d{1,2}[/-]\d{2})\D', dates) # Regexp para fechas con año de dos cifras
  fechas2 = fechas2 + re.findall('(\d{1,2}[/-]\d{1,2}[/-][12]\d{3})', dates) # Regexp para fechas con año de cuatro cifras

  # Creamos dos listas: separando con - y con /
  fechas2A_ = [fecha.split('-') for fecha in fechas2] # Separamos cada fecha por -
  fechas2A_ = pd.DataFrame(fechas2A_, columns = ['month', 'day', 'year']).dropna(axis = 0) # Eliminamos los nulos para quedarnos solo con los que si tienen originalmente -

  fechas2B_ = [fecha.split('/') for fecha in fechas2] # Separamos cada fecha por /
  fechas2B_ = pd.DataFrame(fechas2B_, columns = ['month', 'day', 'year']).dropna(axis = 0) # Eliminamos los nulos para quedarnos solo con los que si tienen originalmente /

  fechas2_ = pd.concat([fechas2A_, fechas2B_]) # Creamos un dataframe con las dos listas anteriores
  fechas2_['year'] = fechas2_['year'].apply(lambda x: int(x) + 1900 if len(str(x)) == 2 else x) # Transformamos aquellas fechas con formato tipo 82 a 1982


  # Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
  fechas3 = re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\D{1,2}\d{1,2}[a-z]{0,2}[.,-/]? [12]\d{3}', dates, flags = re.I) # Regexp

  fechas3_ = [fecha.split(' ') for fecha in fechas3] # Separamos las fechas por espacios
  fechas3_ = pd.DataFrame(fechas3_, columns = ['month', 'day', 'year']) # Creamos dataframe
  fechas3_['month'] = fechas3_['month'].apply(transforma_mes) # Transformamos los meses a número
  fechas3_['day'] = fechas3_['day'].apply(lambda x: x[0:2]) # Eliminamos las comas y puntos que pueden existir

  # 6/2008; 12/2009
  fechas4 = re.findall(r'\D{2,}(\d{1,2}[/-][12]\d{3})', dates, re.I) # Regexp para evitar repetir las fechas de tipo dd/mm/aaaa
  fechas4 = fechas4 + re.findall(r'\d[\n ](\d{1,2}[/-][12]\d{3})', dates) # Agregamos fechas que pueden tener un espacio o salto de línea antes

  fechas4_ = [fecha.split('/') for fecha in fechas4] # Separamos fechas por /
  fechas4_ = pd.DataFrame(fechas4_, columns = ['month', 'year']) # Creamos dataframe
  fechas4_['day'] = 1 # Colocamos el primer día del mes a todas las fechas puesto que no lo tienen específicado

  # 2009, 2010
  fechas5 = re.findall(r'(?:in|quit|since|delivery|nephrectomy|-|:|;)\D{1,2}([12]\d{3})', dates, flags = re.I) # Regexp de años del tipo sine 1900
  fechas5 = fechas5 + re.findall(r'[a-z\.\n]([12]\d{3})', dates, flags = re.I) # fechas de tipo s1900
  fechas5 = fechas5 + re.findall(r'[(]([12]\d{3})[)]', dates, flags = re.I) # Fechas de tipo (1900)
  fechas5 = fechas5 + re.findall(r'([12]\d{3})-[-a-z]', dates, flags = re.I) # Fechas de tipo 1900-- o 1900-new
  fechas5 = fechas5 + re.findall(r'\d{3,} ([12]\d{3})', dates, flags = re.I) # fechas de tipo 123 1900

  fechas5_ = pd.DataFrame(fechas5, columns = ['year']) # Creación de dataframe
  fechas5_['month'] = 1 # Agregamos un 1 al mes
  fechas5_['day'] = 1 # Agregamos un 1 al día 


  # Lista de fechas final
  fechas_ = pd.concat([fechas1_, fechas2_, fechas3_, fechas4_, fechas5_]) # Concatenamos todos los dataframes creados
  fechas_['fecha'] = fechas_.apply(lambda x: str(x['day']) + '/' + str(x['month']) + '/' + str(x['year']), axis = 1) #  Creamos una nueva columna juntando dia mes y año en formato d/m/a

  fechas_['fecha'] = pd.to_datetime(fechas_['fecha'], dayfirst = True) # Transformamos a formato de fecha la columna fecha


  fechas_ = fechas_.sort_values('fecha') # Ordenamos el dataframe de acuerdo a la columna fecha

  fechas_list = [str(fecha) for fecha in fechas_['fecha'].dt.date] # Creamos una lista con la columna fecha del dataframe
  
  print(f"Se tiene un total de {len(fechas_list)} fechas")
  print(f"Lista de fechas ordenadas: {fechas_list}")
  
  return fechas_list # Retornamos la lista

In [51]:
fechas = obtener_fechas('dates')

Archivo cargado correctamente
Se tiene un total de 500 fechas
Lista de fechas ordenadas: ['1971-04-10', '1971-05-18', '1971-07-08', '1971-07-11', '1971-09-12', '1972-01-01', '1972-01-13', '1972-01-26', '1972-05-06', '1972-05-13', '1972-06-10', '1972-06-15', '1972-07-20', '1972-10-04', '1972-11-30', '1973-01-01', '1973-02-01', '1973-02-01', '1973-02-14', '1973-03-01', '1973-03-01', '1973-04-01', '1973-06-01', '1973-07-01', '1973-10-01', '1973-12-01', '1974-01-01', '1974-02-01', '1974-02-24', '1974-03-01', '1974-03-05', '1974-03-06', '1974-04-01', '1974-04-11', '1974-04-12', '1974-05-04', '1974-05-26', '1974-06-01', '1974-06-13', '1974-07-07', '1974-08-01', '1974-09-01', '1974-09-10', '1974-10-10', '1974-10-14', '1975-01-01', '1975-02-28', '1975-03-01', '1975-04-09', '1975-07-01', '1975-07-29', '1975-08-01', '1975-08-18', '1975-09-01', '1975-09-01', '1975-09-27', '1975-11-22', '1975-12-01', '1975-12-14', '1976-02-01', '1976-02-11', '1976-03-01', '1976-03-10', '1976-04-01', '1976-06-01', 

#### Se recuperaron 500 fechas desde 1971 hasta 2016